In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np

import xgboost as xgb
from sklearn.metrics import mean_squared_error, mean_absolute_percentage_error, mean_absolute_error


In [2]:
def predict(data_set_addr: str, model_name: str = "model_rmse.json") -> np.ndarray:
    """
    Function to predict outputs using trained model.
    """


    df = pd.read_csv(data_set_addr)

    
    #Preparing input data to feed into the trained model 
    #(assuming data structure is the same as in "TEST TASK (dataset).csv")
    

    X = df[['total_sessions_day0', 'chapters_finished_day0', 'chapters_opened_day0',
            'chapters_closed_day0', 'diamonds_received_day0', 'diamonds_spent_day0',
            'tickets_spent_day0', 'retained_day1', 'retained_day3', 'retained_day7',
            'app_sub_ltv_day0', 'app_iap_ltv_day0', 'ad_ltv_day0', 'media_source',
            'platform', 'chapters_finished_session9', 'chapters_opened_session9',
            'chapters_closed_session9', 'diamonds_spent_session9','tickets_spent_session9']]

    ind_list = ['0', '1', '3', '7']
    for i in range(1, 4):
        for col_name in ['total_sessions', 'chapters_finished', 'chapters_opened', 'chapters_closed', 
                    'diamonds_received', 'diamonds_spent', 'tickets_spent']:
            X['diff_' +  col_name + '_day' + ind_list[i]] = df[col_name + '_day' + ind_list[i]] - df[col_name + '_day' + ind_list[i-1]]

    ind_list = ['0', '1', '3']
    for i in range(1, 3):
        for col_name in ['app_sub_ltv', 'app_iap_ltv', 'ad_ltv']:
            X['diff_' +  col_name + '_day' + ind_list[i]] = df[col_name + '_day' + ind_list[i]] - df[col_name + '_day' + ind_list[i-1]]

    X = pd.get_dummies(X, columns=['platform', 'media_source'])

    #Now, that the dataset is prepared, let's load the trained model and get the prediction:

    model_xgb = xgb.XGBRegressor()
    model_xgb.load_model(model_name)

    pred = model_xgb.predict(X)

    return pred


Вибір метрики RMSE для цієї моделі був обумовлений декількома факторами, а саме: мінімізація objective функції з частиною RMSE не призведе до сильної упередженості, а також відображає початкову гіпотезу, про зумовленість варіативності вхідних даних випадковими факторами, які можна вважати нормально розподіленими. 

MAPE не є оптимальною метрикою для мінімізації в даному випадку, оскільки цільова змінна часто набуває значень що близькі або дорівнюють нулю, а MAPE обчислюється за формулою:
$$\textrm{MAPE}=\frac{100\%}{n}\sum_{i=1}^n \left| \frac{y_i-\hat{y}_i}{y_i} \right|,$$

де $y_i$ $-$ значення цільової змінної, а $\hat{y}_i$ $-$ передбачене значення. Тобто часто може відбуватися ділення на 0.